In [ ]:
GITHUB_USERNAME = "$GITHUB_USERNAME$"
GITHUB_REF = "$GITHUB_REF$"
NOTEBOOK_TYPE = "$NOTEBOOK_TYPE$"
PYTHON_VERSION = "$PYTHON_VERSION$"
IPYTHON_VERSION = "$IPYTHON_VERSION$"

In [ ]:
import warnings
from pathlib import Path

import requests


warnings.filterwarnings('error', module='davos')

if NOTEBOOK_TYPE == 'colab':
    # utils module doesn't exist on colab VM, so get current version from GitHub
    utils_module = Path('utils.py').resolve()
    response = requests.get(f'https://raw.githubusercontent.com/{GITHUB_USERNAME}/davos/{GITHUB_REF}/tests/utils.py')
    utils_module.write_text(response.text)
    # also need to install davos locally
    from utils import install_davos
    install_davos(source='github', ref=GITHUB_REF, fork=GITHUB_USERNAME)

In [ ]:
import argparse

import davos
from davos.core.exceptions import OnionArgumentError
from davos.core.parsers import pip_parser

from utils import raises, run_tests

In [ ]:
IPYTHON_SHELL = get_ipython()

# tests for `davos.core.parsers`

In [ ]:
def test_args_suppressed_by_default():
    """
    arguments not explicitly passed should not appear in Namespace, 
    *except for "editable" value*. 
    """
    assert pip_parser.argument_default == argparse.SUPPRESS
    
    args = ['foo']
    expected = {
        'editable': False,
        'spec': 'foo'
    }
    parsed_args = vars(pip_parser.parse_args(args))
    assert parsed_args == expected, (
        f"Expected:\n\t{expected}\nFound:\n\t{parsed_args}"
    )

In [ ]:
def test_editable_action():
    """
    Test for EditableAction class -- special Action subclass for 
    '-e/--editable <path/url>' argument that sets 'editable=True' and 
    'spec=<path/url>'.
    """
    # package name should always be assigned to 'spec', 'editable' 
    # should be True/False
    args_regular = ['foo']
    expected_regular = {
        'spec': 'foo',
        'editable': False
    }
    parsed_args_regular = vars(pip_parser.parse_args(args_regular))
    assert parsed_args_regular == expected_regular, (
        f"Result:\n{parsed_args_regular}\nExpected:\n{expected_regular}"
    )
    
    args_editable = '-e bar'.split()
    expected_editable = {
        'spec': 'bar',
        'editable': True
    }
    parsed_args_editable = vars(pip_parser.parse_args(args_editable))
    assert parsed_args_editable == expected_editable, (
        f"Result:\n{parsed_args_editable}\nExpected:\n{expected_editable}"
    )
    
    # one of the two is required...
    args_both = ''.split()
    match_str_both = "Onion comment must specify a package name"
    with raises(OnionArgumentError, match=match_str_both):
        vars(pip_parser.parse_args(args_both))
        
    # and the two are mutually exclusive
    args_neither = 'foo -e bar'.split()
    match_str_neither = "argument -e/--editable: not allowed with argument spec"
    with raises(OnionArgumentError, match=match_str_neither):
        vars(pip_parser.parse_args(args_neither))

In [ ]:
def test_parser_bad_arg_type_raises():
    """passing the wrong Python type for the argument raises an error"""
    args = '--timeout false'.split()
    match_str = "argument --timeout: invalid float value: 'false'"
    with raises(OnionArgumentError, match=match_str):
        pip_parser.parse_args(args)

In [ ]:
def test_unsupported_args_raises():
    """
    should raise an error that lists all unsupported arguments passed, 
    if any
    """
    args = '--not-real -A --rguments 100 --verbose'.split()
    match_str = "argument --not-real: Unrecognized arguments: --not-real -A --rguments"
    with raises(OnionArgumentError, match=match_str):
        pip_parser.parse_args(args)

In [ ]:
run_tests()